# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [3]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/vinit/Desktop/AIM/AIE7/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/vinit/Desktop/AIM/AIE7/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/vinit/Desktop/AIM/AIE7/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


In [4]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'd621db'. Skipping!
Property 'summary' already exists in node 'e3bc71'. Skipping!
Property 'summary' already exists in node '4cfd31'. Skipping!
Property 'summary' already exists in node '937457'. Skipping!
Property 'summary' already exists in node 'f96bd1'. Skipping!
Property 'summary' already exists in node 'd058d2'. Skipping!
Property 'summary' already exists in node '3d8624'. Skipping!
Property 'summary' already exists in node 'd5b724'. Skipping!
Property 'summary' already exists in node '031b50'. Skipping!
Property 'summary' already exists in node '8426c4'. Skipping!
Property 'summary' already exists in node 'a3cb55'. Skipping!
Property 'summary' already exists in node 'a738e8'. Skipping!
Property 'summary' already exists in node 'e6acef'. Skipping!
Property 'summary' already exists in node 'a29f42'. Skipping!


...

Property 'summary_embedding' already exists in node 'e6acef'. Skipping!
Property 'summary_embedding' already exists in node 'a738e8'. Skipping!
Property 'summary_embedding' already exists in node '3d8624'. Skipping!
Property 'summary_embedding' already exists in node '031b50'. Skipping!
Property 'summary_embedding' already exists in node 'a29f42'. Skipping!
Property 'summary_embedding' already exists in node 'd621db'. Skipping!
Property 'summary_embedding' already exists in node 'a3cb55'. Skipping!
Property 'summary_embedding' already exists in node '937457'. Skipping!
Property 'summary_embedding' already exists in node 'e3bc71'. Skipping!
Property 'summary_embedding' already exists in node '8426c4'. Skipping!
Property 'summary_embedding' already exists in node 'd5b724'. Skipping!
Property 'summary_embedding' already exists in node 'd058d2'. Skipping!
Property 'summary_embedding' already exists in node '4cfd31'. Skipping!
Property 'summary_embedding' already exists in node 'f96bd1'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
dataset.to_pandas()

user_input  \
0                     Wut is BBAY 3 for Direct Loanz?   
1   How does clinical work in dentistry programs a...   
2   Are FWS payments required to be made on a paym...   
3   where i find info bout Direct Loan annual loan...   
4   How does the disbursement timing for federal s...   
5   When is a practicum or clinical experience req...   
6   When can practicum or clinical experiences req...   
7   How does disbursement timing for federal stude...   
8   How does the structure of non-term and subscri...   
9   How do the examples in Appendix A and Appendix...   
10  Accorrding to Volume 8, Chapter 3, howw do the...   
11  if i got a medical program with clinical work ...   

                                   reference_contexts  \
0   [non-term (includes clock-hour calendars), or ...   
1   [Inclusion of Clinical Work in a Standard Term...   
2   [Non-Term Characteristics A program that measu...   
3   [both the credit or clock hours and the weeks ...   
4   [<1-hop>\n\nboth the credit or clock hours and...   
5   [<1-hop>\n\nInclusion of Clinical Work in a St...   
6   [<1-hop>\n\nInclusion of Clinical Work in a St...   
7   [<1-hop>\n\nboth the credit or clock hours and...   
8   [<1-hop>\n\nboth the credit or clock hours and...   
9   [<1-hop>\n\nboth the credit or clock hours and...   
10  [<1-hop>\n\nInclusion of Clinical Work in a St...   
11  [<1-hop>\n\nInclusion of Clinical Work in a St...   

                                            reference  \
0   BBAY 3 must be used for monitoring Direct Loan...   
1   Clinical work in dentistry programs can be inc...   
2   No, FWS payments are not required to be made o...   
3   For information on Direct Loan annual loan lim...   
4   In clock-hour or non-term credit-hour programs...   
5   A practicum or clinical experience required fo...   
6   Practicum or clinical experiences required for...   
7   In clock-hour or non-term credit-hour programs...   
8   The structure of non-term and subscription-bas...   
9   The examples in Appendix A illustrate the prin...   
10  Volume 8, Chapter 3 provides guidancce on exce...   
11  if the clinical work in your medical program i...   

                        synthesizer_name  
0   single_hop_specifc_query_synthesizer  
1   single_hop_specifc_query_synthesizer  
2   single_hop_specifc_query_synthesizer  
3   single_hop_specifc_query_synthesizer  
4   multi_hop_abstract_query_synthesizer  
5   multi_hop_abstract_query_synthesizer  
6   multi_hop_abstract_query_synthesizer  
7   multi_hop_abstract_query_synthesizer  
8   multi_hop_specific_query_synthesizer  
9   multi_hop_specific_query_synthesizer  
10  multi_hop_specific_query_synthesizer  
11  multi_hop_specific_query_synthesizer

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [6]:
path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

1102

#### ❓ Question: 

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [9]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="loan_data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="loan_data",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [10]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [12]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [13]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-nano` to avoid using the same model as our judge model.

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

Then we can create a `generate` node!

In [15]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [16]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [18]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [19]:
response = graph.invoke({"question" : "What are the different kinds of loans?"})

In [20]:
response["response"]

'Based on the provided context, the different kinds of loans mentioned are:\n\n1. **Direct Loan** (including subtypes such as Direct Unsubsidized Loan)\n2. **Federal Student Loans** (implied through references to student financial assistance and repayment plans)\n\nThe context specifically discusses the terms and conditions associated with Direct Loans, including the types of repayment plans and loan management strategies, but the primary focus is on the different types of federal student loans under the Direct Loan program.'

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [21]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [22]:
dataset.samples[0].eval_sample.response

"BBAY 3 for Direct Loan refers to a specific monitoring period used for nonstandard, non-term, credit-hour programs that are not subscription-based. It begins on June 1 for a new program and covers the first full academic year during which the student is expected to complete 24 semester hours and 30 weeks of instructional time. BBAY 3 is used to monitor the student's progress and to determine their eligibility for the annual loan limit in such programs."

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [23]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [24]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [25]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

Exception raised in Job[53]: TimeoutError()


{'context_recall': 0.9298, 'faithfulness': 0.9312, 'factual_correctness': 0.7025, 'answer_relevancy': 0.9569, 'context_entity_recall': 0.3075, 'noise_sensitivity_relevant': 0.1879}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [26]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [27]:
adjusted_example_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [28]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [29]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
adjusted_graph_builder.add_edge(START, "retrieve_adjusted")
adjusted_graph = adjusted_graph_builder.compile()

In [30]:
response = adjusted_graph.invoke({"question" : "What are the different kinds of loans?"})
response["response"]

'The provided context mentions the following loan types:\n\n1. Federal PLUS Loans\n2. Federal Family Education Loan (FFEL) Program loans (made under this program before July 1, 2010)\n3. Direct Subsidized Loans\n4. Direct Unsubsidized Loans\n5. Student Direct PLUS Loans\n\nThese are the different kinds of loans referenced in the context.'

In [31]:
import time
import copy

rerank_dataset = copy.deepcopy(dataset)

for test_row in rerank_dataset:
  response = adjusted_graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [32]:
rerank_dataset.samples[0].eval_sample.response

"BBAY 3 for Direct Loan refers to a specific monitoring period used to track a student's progress toward the annual loan limit in a non-term, credit-hour, or nonstandard-term academic program. It begins on June 1 for a new program and corresponds to the first full academic year during which the student is expected to complete 24 semester hours and 30 weeks of instructional time. BBAY 3 is used because the program is non-term, and it helps determine when the student regains eligibility for a new annual loan limit."

In [33]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

In [34]:
result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.7651, 'faithfulness': 0.9271, 'factual_correctness': 0.7133, 'answer_relevancy': 0.9523, 'context_entity_recall': 0.4559, 'noise_sensitivity_relevant': 0.1805}

#### ❓ Question: 

Which system performed better, on what metrics, and why?